In [273]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [274]:
url_basica="https://www.whocc.no/atc_ddd_index/?code="
fin="&showdescription=no"

In [275]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}


In [276]:
#Obtener primer caso
url=url_basica+"A02AB"+fin
page=requests.get(url,headers=headers)

In [277]:
#Obtener el contenido de la pagina
soup=BeautifulSoup(page.content,'html.parser')

In [278]:
# Categorías y códigos
categories = soup.find_all('b')
category_codes = []
category_names = []

for category in categories:
    category_link = category.find('a', href=True)
    if category_link:
        category_code = category_link['href'].split('=')[1].split('&')[0]
        category_name = category_link.text.strip()
        category_codes.append(category_code)
        category_names.append(category_name)


# Extraer la info de la tabla
table = soup.find('table')
rows = table.find_all('tr')

#Pegar todas las filas en una lista
data=[]
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append(cols)

In [279]:
data

[['ATC code', 'Name', 'DDD', 'U', 'Adm.R', 'Note'],
 ['A02AB01', 'aluminium hydroxide', '', '', '', ''],
 ['A02AB02', 'algeldrate', '5', 'g', 'O', ''],
 ['A02AB03', 'aluminium phosphate', '', '', '', ''],
 ['A02AB04', 'dihydroxialumini sodium carbonate', '4', 'g', 'O', ''],
 ['A02AB05', 'aluminium acetoacetate', '', '', '', ''],
 ['A02AB06', 'aloglutamol', '', '', '', ''],
 ['A02AB07', 'aluminium glycinate', '', '', '', ''],
 ['A02AB10', 'combinations', '', '', '', '']]

In [280]:
import numpy as np
df = pd.DataFrame(data, columns=['atc5_code', 'drug_name', 'ddd', 'unit', 'administration', 'note'])
#Incorprar la categoria y el nombre. Cada elemento debe ir en una sola columna
#Eliminar primera fila
df = df.iloc[1:]
#Vacíos como NaN
df = df.replace(r'^\s*$', np.nan, regex=True)
#Rellenar vacíoos de atc5_code y drug_name con el valor de la fila anterior
df['atc5_code'] = df['atc5_code'].fillna(method='ffill')
df['drug_name'] = df['drug_name'].fillna(method='ffill')
df

,atc5_code,drug_name,ddd,unit,administration,note
1,A02AB01,aluminium hydroxide,NaN,NaN,NaN,NaN
2,A02AB02,algeldrate,5,g,O,NaN
3,A02AB03,aluminium phosphate,NaN,NaN,NaN,NaN
4,A02AB04,dihydroxialumini sodium carbonate,4,g,O,NaN
5,A02AB05,aluminium acetoacetate,NaN,NaN,NaN,NaN
6,A02AB06,aloglutamol,NaN,NaN,NaN,NaN
7,A02AB07,aluminium glycinate,NaN,NaN,NaN,NaN
8,A02AB10,combinations,NaN,NaN,NaN,NaN


In [281]:
#Llevr a un dataframe
df = pd.DataFrame(data, columns=['atc5_code', 'drug_name', 'ddd', 'unit', 'administration', 'note'])
#Eliminar primera fila
df = df.iloc[1:]
#Vacíos como NaN
df = df.replace(r'^\s*$', np.nan, regex=True)
#Rellenar vacíoos de atc5_code y drug_name con el valor de la fila anterior
df['atc5_code'] = df['atc5_code'].fillna(method='ffill')
df['drug_name'] = df['drug_name'].fillna(method='ffill')
#Incorprar la categoria y el nombre. Cada elemento debe ir en una sola columna
df['atc1_code'] = df['atc5_code'].apply(lambda x: x[0])
df["atc1_name"] = df["atc1_code"].map(dict(zip(category_codes, category_names)))
df["atc2_code"] = df["atc5_code"].apply(lambda x: x[:3])
df["atc2_name"] = df["atc2_code"].map(dict(zip(category_codes, category_names)))
df["atc3_code"] = df["atc5_code"].apply(lambda x: x[:4])
df["atc3_name"] = df["atc3_code"].map(dict(zip(category_codes, category_names)))
df["atc4_code"] = df["atc5_code"].apply(lambda x: x[:5])
df["atc4_name"] = df["atc4_code"].map(dict(zip(category_codes, category_names)))

#Ordenar las columnas
df = df[['atc1_code', 'atc1_name', 'atc2_code', 'atc2_name', 'atc3_code', 'atc3_name', 'atc4_code', 'atc4_name', 'atc5_code', 'drug_name', 'ddd', 'unit', 'administration', 'note']]
#Minúsculas
df = df.apply(lambda x: x.str.lower() if x.dtype == "object" else x)
df

,atc1_code,atc1_name,atc2_code,atc2_name,atc3_code,atc3_name,atc4_code,atc4_name,atc5_code,drug_name,ddd,unit,administration,note
1,a,alimentary tract and metabolism,a02,drugs for acid related disorders,a02a,antacids,a02ab,aluminium compounds,a02ab01,aluminium hydroxide,NaN,NaN,NaN,NaN
2,a,alimentary tract and metabolism,a02,drugs for acid related disorders,a02a,antacids,a02ab,aluminium compounds,a02ab02,algeldrate,5,g,o,NaN
3,a,alimentary tract and metabolism,a02,drugs for acid related disorders,a02a,antacids,a02ab,aluminium compounds,a02ab03,aluminium phosphate,NaN,NaN,NaN,NaN
4,a,alimentary tract and metabolism,a02,drugs for acid related disorders,a02a,antacids,a02ab,aluminium compounds,a02ab04,dihydroxialumini sodium carbonate,4,g,o,NaN
5,a,alimentary tract and metabolism,a02,drugs for acid related disorders,a02a,antacids,a02ab,aluminium compounds,a02ab05,aluminium acetoacetate,NaN,NaN,NaN,NaN
6,a,alimentary tract and metabolism,a02,drugs for acid related disorders,a02a,antacids,a02ab,aluminium compounds,a02ab06,aloglutamol,NaN,NaN,NaN,NaN
7,a,alimentary tract and metabolism,a02,drugs for acid related disorders,a02a,antacids,a02ab,aluminium compounds,a02ab07,aluminium glycinate,NaN,NaN,NaN,NaN
8,a,alimentary tract and metabolism,a02,drugs for acid related disorders,a02a,antacids,a02ab,aluminium compounds,a02ab10,combinations,NaN,NaN,NaN,NaN
